In [0]:
df_silver = spark.table("workspace.bronze.crm_prd_info")

In [0]:
df_silver.display()

In [0]:
df_silver.printSchema()

In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
# 1. Define the Window (The "OVER" clause in your SQL)
window_spec = Window.partitionBy("prd_key").orderBy("prd_start_dt")

df_clean = (
    df_silver
    # 2. Apply the LEAD function, subtract 1 day, and cast as date
    .withColumn(
        "prd_end_dt", 
        F.date_sub(F.lead("prd_start_dt").over(window_spec), 1).cast(DateType())
    )
)

In [0]:
import pyspark.sql.functions as F

df_silver = (
    df_silver
    # 1. Parentheses fixed: (trim(upper(col))) goes FIRST, then the 1, 5
    .withColumn('cat_id', F.regexp_replace(F.substring(F.trim(F.upper(F.col("prd_key"))), 1, 5), "-", "_"))
    
    # 2. Parentheses fixed, and used 100 instead of F.length() for simplicity
    .withColumn('prd_key', F.substring(F.trim(F.upper(F.col("prd_key"))), 7, 100))
)

display(df_silver)

In [0]:
df_silver.summary("count").show()

In [0]:

df_silver = (
    df_silver
    .withColumn(
        "prd_cost", 
        F.when(F.col("prd_cost").isNull(), "0")
         .otherwise(F.col("prd_cost")) # Keep the existing cost if it's not null!
    )
)

In [0]:
df_unique = df_silver.select("prd_line").distinct()

display(df_unique)

In [0]:

df_silver = (
    df_silver
    # 1. FIRST: Force the column to be uppercase and remove any invisible spaces
    .withColumn("prd_line", F.trim(F.upper(F.col("prd_line"))))
    
    # 2. THEN: Run your translation logic on the freshly cleaned letters
    .withColumn(
        "prd_line",
        F.when(F.col("prd_line") == "R", "Road")
         .when(F.col("prd_line") == "S", "Other Sales")
         .when(F.col("prd_line") == "M", "Mountain")
         .when(F.col("prd_line") == "T", "Touring") # (Or "B", depending on your data)
         .otherwise("n/a")
    )
)

display(df_silver)

In [0]:
df_silver.filter(F.col("prd_line") == "n/a").count()

In [0]:
from pyspark.sql.types import StringType, DateType
df_silver = df_silver.withColumn("prd_start_dt", F.col("prd_start_dt").cast(DateType()))

In [0]:
df_silver.display()

In [0]:
RENAME_MAP = {
    "prd_id": "product_id",
    "cat_id": "category_id",
    "prd_key": "product_number",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date"
}
for old_name, new_name in RENAME_MAP.items():
    df_silver = df_silver.withColumnRenamed(old_name, new_name)

In [0]:
df_silver.limit(10).display()

In [0]:
df_silver.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("workspace.silver.crm_products")